# Data pre-processing notebook
### Dropping non-songs, encoding categorical variables

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("./Data/tracks.csv")
df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


Data cleanup

In [4]:
import category_encoders as ce

def clean_data(df):
    # Drop the id column, name, release_date
    ### Engineer year feature
    x = df.copy()
    drop_cols = ['id', 'name', 'release_date', 'id_artists', 'artists']
    x = x.drop(labels=drop_cols, axis=1)
    
    # Drop zero tempo songs
    x = x[x['tempo']!=0]
    
    # Do one-hot encoding for key
    ohe = ce.one_hot.OneHotEncoder(cols=['key'])
    x = ohe.fit_transform(x)
    
    # Do ordinal encoding for artists
    #oe = ce.ordinal.OrdinalEncoder()
    #x_oe = oe.fit_transform(x[['artists']])
    #x['artists'] = x_oe
    
    # Normalize numeric features
    num_features = ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature']
    
    for i in num_features:
        col_mean = x[i].values.mean()
        col_var = np.var(df[i])
        col_norm = [(o - col_mean) / col_var**(1/2) for o in x[i].values]
        x[i] = col_norm
    
    return x


/opt/anaconda3/envs/Unit_4/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Old df shape: (586672, 20)
Old df columns: ['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists', 'id_artists', 'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

Cleaned df shape: (586344, 26)
Clean df columns: ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']


In [ ]:

clean_df = clean_data(df)
print(f'Old df shape: {df.shape}\nOld df columns: {[i for i in df.columns]}\n\n'
        f'Cleaned df shape: {clean_df.shape}\nClean df columns: {[i for i in clean_df.columns]}')

In [5]:
clean_df.to_csv('./Data/songs_cleaned.csv')